In [1]:
import dash
from dash.dependencies import Input, Output
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
import flask
from flask_cors import CORS
import os
import numpy as np

from sklearn.metrics.pairwise import pairwise_distances
from scipy.stats import entropy
import networkx as nx

In [2]:
app = dash.Dash('drug-discovery')
server = app.server

In [3]:
# Biz name : <name>

# Star rating : <star rating>

# Main topics : < Topic names >

# Relevant words : < [Wordlist]>


In [4]:
df = pd.read_csv('./dash-drug-discovery-demo/small_molecule_drugbank.csv').drop(['Unnamed: 0'],axis=1)
df.DESC[0] = "Kunal \n 4.5 \n sdfndsfgdk"

/anaconda/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [5]:
df.head()

,NAME,PAGE,IMG_URL,LOGP,PKA,MW,FORM,SOL,DESC
0,Cyclacillin,https://www.drugbank.ca/drugs/DB01000,https://www.drugbank.ca/structures/DB01000/thu...,1.31,3.30,341.4260,C15H23N3O4S,1.9000,Kunal \n 4.5 \n sdfndsfgdk
1,Salbutamol,https://www.drugbank.ca/drugs/DB01001,https://www.drugbank.ca/structures/DB01001/thu...,1.40,10.12,239.3107,C13H21NO3,2.1500,"Salbutamol is a short-acting, selective beta2-..."
2,Levobupivacaine,https://www.drugbank.ca/drugs/DB01002,https://www.drugbank.ca/structures/DB01002/thu...,3.60,13.62,288.4277,C18H28N2O,0.0977,Levobupivacaine is an amino-amide local anaest...
3,Cromoglicic acid,https://www.drugbank.ca/drugs/DB01003,https://www.drugbank.ca/structures/DB01003/thu...,1.92,1.77,468.3665,C23H16O11,0.0358,A chromone complex that acts by inhibiting the...
4,Ganciclovir,https://www.drugbank.ca/drugs/DB01004,https://www.drugbank.ca/structures/DB01004/thu...,-1.66,10.16,255.2306,C9H13N5O4,11.5000,An acyclovir analog that is a potent inhibitor...


In [6]:
df_yelp = pd.read_feather('./df_final_doc2topics.feather')
df_yelp.drop(['business_id'], axis=1, inplace=True)
data = df_yelp.drop(['name', 'is_strip', 'stars'], axis=1).as_matrix()

In [7]:
df = df.iloc[:df_yelp.shape[0]]  # reduce num of rows to be same as df_yelp

In [8]:
# dont worry about this cell
if 'DYNO' in os.environ:
    app.scripts.append_script({
        'external_url': 'https://cdn.rawgit.com/chriddyp/ca0d8f02a1659981a0ea7f013a378bbd/raw/e79f3f789517deec58f41251f7dbb6bee72c44ab/plotly_ga.js'
    })
    
BACKGROUND = 'rgb(230, 230, 230)'

COLORSCALE = [ [0, "rgb(244,236,21)"], [0.3, "rgb(249,210,41)"], [0.4, "rgb(134,191,118)"],
                [0.5, "rgb(37,180,167)"], [0.65, "rgb(17,123,215)"], [1, "rgb(54,50,153)"] ]

In [9]:
def jensen_shannon(_P, _Q):
    _M = 0.5 * (_P + _Q)
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))

In [10]:
# Pairwise Jensen-Shannon distance between each pair of observations based on the 18 topic-probabilities
pairwise_dist = pairwise_distances(X=data, metric=jensen_shannon)

In [11]:
# find the topic most closely related to each restaurant
topic_closest_ind = np.argmax(data, axis=1)

topic_names_ord = ['Cost & Quality', 'Bars', 'Casino Hotel', 'Fine Dining', 'Asian', 'Pizza', 'Steakhouse', 
                   'Italian', 'Coffee Shop', 'High Customer Satisfaction', 'Night Club', 'Wait Time', 'Mexican', 
                   'Lunch', 'Sushi', 'Fast Food', 'Breakfast', 'Low Customer Satisfaction']

# names of topics most closely related to each restaurant (ordered by the order of restaurants in df)
topic_closest = [topic_names_ord[ind] for ind in topic_closest_ind]

In [12]:
threshold2k ={
    0.55: 0.7,
    0.56: 0.9,
    0.57: 0.3,
    0.58: 5,
    0.59: 2,
    0.6: 5,
    0.61: 5,
    0.62: 5
}

In [13]:
# arbitrary threshold for deciding whether 2 observations are 'similar' or not
threshold_all = [0.55, 0.56, 0.57, 0.58, 0.59, 0.6, 0.61, 0.62]
def th_mark(x):
    if x==np.min(threshold_all):
        return 'Low'
    elif x==np.max(threshold_all):
        return 'High'
    else:
        return ''
    
threshold_mark = {str(th):th_mark(th) for th in threshold_all}
adjacency = [np.where(pairwise_dist > threshold, 1, 0) for threshold in threshold_all]

In [14]:
# map threshold value to adjacency matrix
thresh_to_adj = {thresh: adj for thresh, adj in zip(threshold_all, adjacency)}

In [15]:
def create_graph(adj):
    # input: adjaccency matrix
    # returns a graph with the isolates removed
    G = nx.from_numpy_matrix(adj)
    isolates = list(nx.isolates(G))
    G.remove_nodes_from(isolates)
    return G

In [16]:
# map threshold value to graph
thresh_to_graph = {thresh: create_graph(adj) for thresh, adj in zip(threshold_all, adjacency)}

In [17]:
# extract node positions
fruchterman_iter = 1000

# map threshold values to positions of nodes
thresh_to_pos = {}

for thresh in thresh_to_graph:
    graph = nx.fruchterman_reingold_layout(thresh_to_graph[thresh], k = threshold2k[thresh], iterations=fruchterman_iter)
    thresh_to_pos[thresh] = graph

In [18]:
thresh_to_XnYn = {}
for thresh in thresh_to_pos:
    pos = thresh_to_pos[thresh]
    # define lists of node coordinates
    Xn_strip = [pos[k][0] for k in sorted(pos.keys()) if k in df_yelp.index[df_yelp.is_strip == True]]
    Yn_strip = [pos[k][1] for k in sorted(pos.keys()) if k in df_yelp.index[df_yelp.is_strip == True]]
    Xn_notstrip = [pos[k][0] for k in sorted(pos.keys()) if k in df_yelp.index[df_yelp.is_strip == False]]
    Yn_notstrip = [pos[k][1] for k in sorted(pos.keys()) if k in df_yelp.index[df_yelp.is_strip == False]]
    thresh_to_XnYn[thresh] = (Xn_strip, Yn_strip, Xn_notstrip, Yn_notstrip)

In [19]:
# concatenating the 2 dataframes
df = pd.concat([df, df_yelp], axis=1)

In [20]:
df.stars.value_counts()//8

4.0    14
4.5     9
3.5     3
3.0     1
2.5     0
5.0     0
Name: stars, dtype: int64

In [21]:

df.head().T

,0,1,2,3,4
NAME,Cyclacillin,Salbutamol,Levobupivacaine,Cromoglicic acid,Ganciclovir
PAGE,https://www.drugbank.ca/drugs/DB01000,https://www.drugbank.ca/drugs/DB01001,https://www.drugbank.ca/drugs/DB01002,https://www.drugbank.ca/drugs/DB01003,https://www.drugbank.ca/drugs/DB01004
IMG_URL,https://www.drugbank.ca/structures/DB01000/thu...,https://www.drugbank.ca/structures/DB01001/thu...,https://www.drugbank.ca/structures/DB01002/thu...,https://www.drugbank.ca/structures/DB01003/thu...,https://www.drugbank.ca/structures/DB01004/thu...
LOGP,1.31,1.4,3.6,1.92,-1.66
PKA,3.3,10.12,13.62,1.77,10.16
MW,341.426,239.311,288.428,468.366,255.231
FORM,C15H23N3O4S,C13H21NO3,C18H28N2O,C23H16O11,C9H13N5O4
SOL,1.9,2.15,0.0977,0.0358,11.5
DESC,Kunal \n 4.5 \n sdfndsfgdk,"Salbutamol is a short-acting, selective beta2-...",Levobupivacaine is an amino-amide local anaest...,A chromone complex that acts by inhibiting the...,An acyclovir analog that is a potent inhibitor...
name,Jaburritos,Le Thai,Sugar Factory,Rollin' Smoke BBQ,PublicUs


In [22]:
df['temp_id'] = df.index

In [23]:
# map threshold value to a list of nodes left over in the graph after isolate removal
thresh_to_nodenums = {}
for thresh in thresh_to_graph:
    graph = thresh_to_graph[thresh]
    nodenums = list(graph.nodes())
    thresh_to_nodenums[thresh] = nodenums

In [24]:
# format of thresh_to_XnYn's output:
# thresh_to_XnYn[thresh] = (Xn_strip, Yn_strip, Xn_notstrip, Yn_notstrip)

stacked_df = []
for thresh in thresh_to_XnYn:
    # for each threshold, create a copy of df
    df_temp = df.copy()
    
    # positions of nodes for the graph
    Xn_strip, Yn_strip, Xn_notstrip, Yn_notstrip = thresh_to_XnYn[thresh]
    # nodes left after removing isolates
    nodenums = thresh_to_nodenums[thresh]
    
    df_temp['threshold'] = thresh
    # initialize Xn and Yn
    df_temp['Xn'] = np.nan
    df_temp['Yn'] = np.nan
    # fill in X and Y positions of nodes for non-isolate nodes
    df_temp.loc[(df_temp.is_strip == True) & (df_temp.temp_id.isin(nodenums)), 'Xn'] = Xn_strip
    df_temp.loc[(df_temp.is_strip == True) & (df_temp.temp_id.isin(nodenums)), 'Yn'] = Yn_strip
    df_temp.loc[(df_temp.is_strip == False) & (df_temp.temp_id.isin(nodenums)), 'Xn'] = Xn_notstrip
    df_temp.loc[(df_temp.is_strip == False) & (df_temp.temp_id.isin(nodenums)), 'Yn'] = Yn_notstrip
    
    stacked_df.append(df_temp)
    

In [25]:
# vertically stack the dataframes in stacked_df
df_final = pd.concat(stacked_df, axis=0, ignore_index=True)

In [26]:
df_final.shape

(1896, 34)

In [27]:
df_final.loc[0,'Xn']

-0.12221802613447123

In [28]:
df_final.head()

,NAME,PAGE,IMG_URL,LOGP,PKA,MW,FORM,SOL,DESC,name,...,Topic13,Topic14,Topic15,Topic16,Topic17,Topic18,temp_id,threshold,Xn,Yn
0,Cyclacillin,https://www.drugbank.ca/drugs/DB01000,https://www.drugbank.ca/structures/DB01000/thu...,1.31,3.30,341.4260,C15H23N3O4S,1.9000,Kunal \n 4.5 \n sdfndsfgdk,Jaburritos,...,0.102890,0.090591,0.011609,0.054163,0.013609,0.022229,0,0.55,-0.122218,0.019718
1,Salbutamol,https://www.drugbank.ca/drugs/DB01001,https://www.drugbank.ca/structures/DB01001/thu...,1.40,10.12,239.3107,C13H21NO3,2.1500,"Salbutamol is a short-acting, selective beta2-...",Le Thai,...,0.016207,0.006142,0.006792,0.173229,0.003961,0.005508,1,0.55,0.783943,-0.314338
2,Levobupivacaine,https://www.drugbank.ca/drugs/DB01002,https://www.drugbank.ca/structures/DB01002/thu...,3.60,13.62,288.4277,C18H28N2O,0.0977,Levobupivacaine is an amino-amide local anaest...,Sugar Factory,...,0.028241,0.012212,0.009585,0.318069,0.021889,0.019387,2,0.55,NaN,NaN
3,Cromoglicic acid,https://www.drugbank.ca/drugs/DB01003,https://www.drugbank.ca/structures/DB01003/thu...,1.92,1.77,468.3665,C23H16O11,0.0358,A chromone complex that acts by inhibiting the...,Rollin' Smoke BBQ,...,0.000012,0.041125,0.000012,0.033540,0.616419,0.107080,3,0.55,0.335476,-0.078127
4,Ganciclovir,https://www.drugbank.ca/drugs/DB01004,https://www.drugbank.ca/structures/DB01004/thu...,-1.66,10.16,255.2306,C9H13N5O4,11.5000,An acyclovir analog that is a potent inhibitor...,PublicUs,...,0.013026,0.085354,0.005334,0.101793,0.177481,0.037548,4,0.55,NaN,NaN


In [29]:
def update_slider_mark(slider_mark, font_size):
    # update display style of position markers for the slider
    slider_mark_updated = {}
    for position in slider_mark:
        slider_mark_updated[position] = {
            'label': slider_mark[position],
            'style': {'fontSize':font_size, 'font-family': 'Arial'}
        }
    return slider_mark_updated

threshold_mark_updated = update_slider_mark(threshold_mark, 15)

In [30]:
del df
df = df_final

In [31]:
# figure data is the data object we pass into figure function 
# molecules will be the selected business
# change this function for our needs

def add_markers(selected_threshold, df, molecules, plot_type = 'scatter' ):
    indices = []
    drug_data = df
    for m in molecules:
        # this is the text attribute of data object 
        hover_text = drug_data.NAME.tolist()
        for i in range(len(hover_text)):
            if m == hover_text[i]:
                indices.append(i)
                print(i)
    print(indices)
    trace_markers = []
    for point_number in indices:
        print(drug_data.loc[point_number,'Xn'])
        print(drug_data.loc[point_number,'Xn'].tolist())
        trace = dict(
            x = [drug_data.loc[point_number,'Xn']],
            y = [drug_data.loc[point_number,'Yn']],
            marker = dict(
                color = 'black',
                size = 10,
                opacity = 0.6,
                symbol = 'cross'
            ),
            type = plot_type
        )
        trace_markers.append(trace)
    print(trace_markers)   
    Xn_strip, Yn_strip, Xn_notstrip, Yn_notstrip = thresh_to_XnYn[selected_threshold]

    # define a trace for plotly
    trace_nodes1 = dict(type='scatter', 
                        x=Xn_strip, 
                        y=Yn_strip,
                        mode='markers',
                        marker=dict(symbol='dot', 
                                    size=10, color='rgb(255,0,0)'),
                        name='On The Strip',
                        showlegend=True, 
                        text = [],
                        hoverinfo='text',
                        visible=True)
    trace_nodes2 = dict(type='scatter', 
                        x=Xn_notstrip, 
                        y=Yn_notstrip,
                        mode='markers',
                        marker=dict(symbol='dot', 
                                    size=10, color='rgb(0, 0, 255)'),
                        name='Not on The Strip',
                        showlegend=True, 
                        text = [],
                        hoverinfo='text',
                        visible=True)

    # ADD THIS LATER
#     # Add labels for nodes
#     for index, row in df.iterrows():
#         # for strip restaurants
#         if index in df.index[df.is_strip == True]:
#             node_info = df.name.iloc[index] + ', ' + str(df.stars.iloc[index]) + '/5, Related to: ' + topic_closest[index]
#             trace_nodes1['text'].append(node_info)
#         # for non strip restaurants
#         if index in df.index[df.is_strip == False]:
#             node_info = df.name.iloc[index] + ', ' + str(df.stars.iloc[index]) + '/5, Related to: ' + topic_closest[index]
#             trace_nodes2['text'].append(node_info)
        
    
    # record the coordinates of the ends of edges
    Xe = []
    Ye = []
    G = thresh_to_graph[selected_threshold]
    for e in G.edges():
        pos = thresh_to_pos[selected_threshold]
        Xe.extend([pos[e[0]][0], pos[e[1]][0], None])
        Ye.extend([pos[e[0]][1], pos[e[1]][1], None])

    # trace_edges defines the graph edges as a trace of type scatter (line)
    trace_edges=dict(type='scatter',
                     mode='lines',
                     x=Xe,
                     y=Ye,
                     line=dict(width=0.1, color='rgb(51, 51, 51)'),
                     hoverinfo='none', showlegend=False)
    traces = [trace_nodes1] + [trace_nodes2] + [trace_edges]+trace_markers
    return traces

# change this function 
def scatter_plot_3d(selected_threshold, xlabel='LogP', ylabel='pkA', plot_type='scatter', markers=[]):
    df_for_plot = df.copy()
    df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
    df_for_plot = df_for_plot.reset_index().drop('index',axis=1)
    x= df_for_plot['Xn'] 
    y= df_for_plot['Yn']
    size= df_for_plot['MW']
    color= df_for_plot['MW']
    def axis_template_2d(title):
        return dict(
            xgap = 10, ygap = 10,
            backgroundcolor = BACKGROUND,
            gridcolor = 'rgb(255, 255, 255)',
            title = title,
            zerolinecolor = 'rgb(255, 255, 255)',
            color = '#444'
        )
    # change data
    data = [dict(
        x = x,
        y = y,
        mode = 'markers',
        marker = dict(
                colorscale = COLORSCALE,
                colorbar = dict( title = "Molecular<br>Weight" ),
                line = dict( color = '#444' ),
                reversescale = True,
                sizeref = 45,
                sizemode = 'diameter',
                opacity = 0.7,
                size = size,
                color = color,
            ),
        text = df_for_plot['NAME'],
        type = plot_type,
    ) ]
    # change layour
    layout = dict(
        font = dict( family = 'Raleway' ),
        hovermode = 'skip',
        margin = dict( r=20, t=0, l=0, b=0 ),
        showlegend = False
    )
    # change scatter
    if plot_type in ['scatter']:
        layout['xaxis'] = axis_template_2d(xlabel)
        layout['yaxis'] = axis_template_2d(ylabel)
        layout['plot_bgcolor'] = BACKGROUND
        layout['paper_bgcolor'] = BACKGROUND
        
    # keep this
    
    data = data + add_markers(selected_threshold, df_for_plot, markers, plot_type = plot_type )

    return dict(data=data, layout=layout)

selected_threshold = threshold_all[0]
FIGURE = scatter_plot_3d(selected_threshold)
df_for_plot = df.copy()
df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
df_for_plot = df_for_plot.reset_index().drop('index',axis=1)
STARTING_DRUG = df_for_plot.loc[0,'NAME']
DRUG_DESCRIPTION = df_for_plot.loc[df_for_plot['NAME'] == STARTING_DRUG]['DESC'].iloc[0]
STAR_RATING =df_for_plot.loc[df_for_plot['NAME'] == STARTING_DRUG]['stars'].iloc[0]
DRUG_IMG = df_for_plot.loc[df_for_plot['NAME'] == STARTING_DRUG]['IMG_URL'].iloc[0]

[]
[]


In [ ]:
app.layout = html.Div([
    # Row 1: Header and Intro text
    html.Div([
        html.Div([
            html.Div([
                html.P('HOVER over a drug in the graph to the right to see its structure to the left.'),
                html.P('SELECT a drug in the dropdown to add it to the drug candidates at the bottom.')
            ], style={'margin-left': '10px'}),
            dcc.Dropdown(id='chem_dropdown',
                        multi=True,
                        value=[ STARTING_DRUG ],
                        options=[{'label': i, 'value': i} for i in df_for_plot['NAME'].tolist()]),
            ], className='twelve columns' )

    ], className='row' ),

    # Row 2: Hover Panel and Graph
    html.Div([
        html.Div([

            html.Img(id='chem_img', src=DRUG_IMG ),

            html.Br(),

            html.A(STARTING_DRUG,
                  id='chem_name',
                  href="https://www.drugbank.ca/drugs/DB01002",
                  target="_blank"),

            html.P(DRUG_DESCRIPTION,
                  id='chem_desc',
                  style=dict( maxHeight='400px', fontSize='12px' )),
            html.Br(),
            html.P(STAR_RATING,
                  id='star_rating',
                  style=dict( maxHeight='400px', fontSize='12px' ))

        ], className='three columns', style=dict(height='300px')),

        html.Div([
            dcc.Graph(id='clickable-graph',
                      style=dict(width='700px'),
                      hoverData=dict(points=[dict(pointNumber=0)] ),
                      figure=FIGURE ),

        ], className='nine columns', style=dict(textAlign='center')),
    html.Div([
    html.H2('Similarity Cutoff'),
    dcc.Slider(
        id='threshold-slider',
        min=min(threshold_all),
        max=max(threshold_all),
        value=selected_threshold,
        step=None,
        marks=threshold_mark_updated
    ),
    ], style={'width': '47%','marginBottom': 0, 'marginTop': 0, 'marginLeft':'auto', 'marginRight':'auto',
              'fontSize':12, 'font-family': 'Arial'}
    )
    ], className='row' ),

], className='container')


@app.callback(
    Output('clickable-graph', 'figure'),
    [Input('chem_dropdown', 'value'), Input('threshold-slider', 'value')])
def highlight_molecule(chem_dropdown_values, selected_threshold):
    return scatter_plot_3d(selected_threshold=selected_threshold, 
                           markers = chem_dropdown_values, plot_type = 'scatter')



def dfRowFromHover(hoverData,selected_threshold):
    ''' Returns row for hover point as a Pandas Series '''
    if hoverData is not None:
        if 'points' in hoverData:
            firstPoint = hoverData['points'][0]
            if 'pointNumber' in firstPoint:
                point_number = firstPoint['pointNumber']
#                 molecule_name = str(FIGURE['data'][0]['text'][point_number]).strip()

                df_for_plot = df.copy()
                df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
                df_for_plot = df_for_plot.reset_index().drop('index',axis=1)
                molecule_name = df_for_plot.loc[point_number,'NAME']
                return df_for_plot.loc[df_for_plot['NAME'] == molecule_name]
    return pd.Series()


@app.callback(
    dash.dependencies.Output('chem_dropdown', 'options'),
    [dash.dependencies.Input('threshold-slider', 'value')])
def set_dropdown_options(selected_threshold):
    df_for_plot = df.copy()
    df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
    df_for_plot = df_for_plot.reset_index().drop('index',axis=1)
    return [{'label': i, 'value': i} for i in df_for_plot['NAME'].tolist()]

@app.callback(
    dash.dependencies.Output('chem_dropdown', 'value'),
    [dash.dependencies.Input('chem_dropdown', 'options')])
def set_dropdown_value(available_options):
    return available_options[0]['value']

@app.callback(
    Output('chem_name', 'children'),
    [Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def return_molecule_name(hoverData,selected_threshold):
    if hoverData is not None:
#         print(hoverData)
        if 'points' in hoverData:
            firstPoint = hoverData['points'][0]
            if 'pointNumber' in firstPoint:
                point_number = firstPoint['pointNumber']
                df_for_plot = df.copy()
                df_for_plot = df_for_plot.loc[(df_for_plot.Xn.isnull() == False) & (df_for_plot.Yn.isnull() == False) & (df_for_plot.threshold == selected_threshold), :]
                df_for_plot = df_for_plot.reset_index().drop('index',axis=1)

                try:
                    molecule_name = df_for_plot.loc[point_number,'NAME']
                except KeyError:
                    return None
#                 print('pointnumbr',FIGURE['data'][0]['text'])
#                 molecule_name = str(FIGURE['data'][0]['text'][point_number]).strip()
    
                return molecule_name


@app.callback(
    dash.dependencies.Output('chem_name', 'href'),
    [dash.dependencies.Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def return_href(hoverData,selected_threshold):
    row = dfRowFromHover(hoverData,selected_threshold)
    if row.empty:
        return
    datasheet_link = row['PAGE'].iloc[0]
    return datasheet_link


@app.callback(
    Output('chem_img', 'src'),
    [Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def display_image(hoverData,selected_threshold):
    row = dfRowFromHover(hoverData,selected_threshold)
    if row.empty:
        return
    img_src = row['IMG_URL'].iloc[0]
    return img_src


@app.callback(
    Output('chem_desc', 'children'),
    [Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def display_molecule(hoverData,selected_threshold):
    row = dfRowFromHover(hoverData,selected_threshold)
    if row.empty:
        return
    description = row['DESC'].iloc[0]
    return description

@app.callback(
    Output('star_rating', 'children'),
    [Input('clickable-graph', 'hoverData'),Input('threshold-slider', 'value')])
def display_star(hoverData,selected_threshold):
    row = dfRowFromHover(hoverData,selected_threshold)
    if row.empty:
        return
    star = row['stars'].iloc[0]
    return star

In [ ]:
external_css = ["https://cdnjs.cloudflare.com/ajax/libs/skeleton/2.0.4/skeleton.min.css",
                "//fonts.googleapis.com/css?family=Raleway:400,300,600",
                "//fonts.googleapis.com/css?family=Dosis:Medium",
                "https://cdn.rawgit.com/plotly/dash-app-stylesheets/0e463810ed36927caf20372b6411690692f94819/dash-drug-discovery-demo-stylesheet.css"]


for css in external_css:
    app.css.append_css({"external_url": css})


if __name__ == '__main__':
    app.run_server()


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [09/May/2018 21:21:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:37] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:37] "POST /_dash-update-component HTTP/1.1" 2

[]
[]


127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 21:21:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.

...

127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:46] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [09/May/2018 22:02:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:48] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [09/May/2018 22:02:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:02:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:00] "POST /_dash-update-component HTTP/1.1" 200 -
127.

127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2018 22:03:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.